## Intro

Polars is a popular single machine DataFrame library powered by an OLAP Query Engine. Beginning in the v1.3 release, Polars can now leverage NVIDIA GPUs for even higher performance through its GPU engine (powered by RAPIDS cuDF).

Designed to make processing 10-100+ GBs of data feel interactive with just a single GPU, this new engine is built directly into the Polars Lazy API – just pass <font color="#76B900">**engine="gpu"**</font> to the `collect` operation.

The GPU engine fully utilizes the Polars optimizer to ensure efficient execution and minimal memory usage, is compatible with the ecosystem of tools built for Polars, and has graceful CPU fallback for unsupported queries.

This notebook is a short introduction to the Polars GPU engine, powered by cuDF.

In [ ]:
!nvidia-smi | head

In [ ]:
!lscpu | grep -E "Model|Socket"

# Setup

## Installing the Polars GPU Engine

Install Polars with GPU Engine by using a feature flag in the standard pip install command  `pip install polars[gpu] --extra-index-url=https://pypi.nvidia.com`.

In [ ]:
!pip install polars[gpu] --extra-index-url=https://pypi.nvidia.com

If you aren't running in Google Colab, please install by following the instructions in the README.

## Download the Data

We'll be working with a roughly 22GB dataset of [simulated financial transactions from Kaggle](https://www.kaggle.com/datasets/conorsully1/simulated-transactions) by default. If you're running this notebook on Google Colab using the T4 GPU in the Colab free tier, make sure you set `GOOGLE_COLAB=True` below to download a smaller version of this dataset (about 20% of the size) to fit on the relatively weaker CPU and GPU.

We're downloading a copy of this dataset from a GCS bucket hosted by NVIDIA to provide faster download speeds. We'll start by downloading the data. This should take about 30 seconds.

In [ ]:
if GOOGLE_COLAB:
    !wget https://storage.googleapis.com/rapidsai/polars-demo/transactions-t4-20.parquet -O transactions.parquet
else:
    !wget https://storage.googleapis.com/rapidsai/polars-demo/transactions.parquet -O transactions.parquet

!wget https://storage.googleapis.com/rapidsai/polars-demo/rainfall_data_2010_2020.csv

## Data License and Terms
As this dataset originates from Kaggle, it's governed by a Kaggle dataset-specific license and terms of use.

> ### CC0 1.0 Universal
> **No Copyright**

>The person who associated a work with this deed has dedicated the work to the public domain by waiving all of his or her rights to the work worldwide under copyright law, including all related and neighboring rights, to the extent allowed by law.
You can copy, modify, distribute and perform the work, even for commercial purposes, all without asking permission. See Other Information below.

> **Other Information**

>In no way are the patent or trademark rights of any person affected by CC0, nor are the rights that other persons may have in the work or in how the work is used, such as publicity or privacy rights.
Unless expressly stated otherwise, the person who associated a work with this deed makes no warranties about the work, and disclaims liability for all uses of the work, to the fullest extent permitted by applicable law.
When using or citing the work, you should not imply endorsement by the author or the affirmer.


# Install Plotting Dependencies
To use the built-in data visualization capabilities of Polars v1.5, you'll need to install a few additional dependencies.

In [ ]:
!pip install hvplot jupyter_bokeh holoviews==1.19

# Getting Started

With this dataset, we expect to see some meaningful performance benefits using NVIDIA GPUs for computationally heavy queries and limited benefits for basic queries that are primarily bottlenecked by reading data.

To begin, let's use Polars to read the parquet file and look at the schema and first few rows.

In [ ]:
import polars as pl
from polars.testing import assert_frame_equal

In [ ]:
pl.__version__ # Make sure the version is `1.5.x`; If not restart the session.

In [ ]:
transactions = pl.scan_parquet("transactions.parquet")

In [ ]:
transactions.collect_schema()

In [ ]:
transactions.head(5).collect()

This dataset has a mix of data types (strings, dates, integers, and floats), some missing values, and it looks like the dates were actually parsed as strings. We'll need to handle that during some of our analysis.

# Our first GPU operation: Total aggregate transaction amount

With Polars, to calculate the total spending in of all transactions we simply `sum` the `AMOUNT` column:

In [ ]:
transactions.select(pl.col("AMOUNT").sum()).collect()

Looks like we're handling a high total transaction volume. Let's run the same query on the GPU.

To use a GPU, we just need to tell Polars to use the GPU engine when we call `collect` by passing `engine="gpu"` as a parameter.


```python
transactions.select(pl.col("AMOUNT").sum()).collect(engine="gpu")
```

The default configuration is likely the right choice for most workloads, but sometimes we want more control. We can provide more detailed configuration options (such as which device to run on and a variety of other options) by passing a Polars `GPUEngine` object to the `engine` parameter instead.


In this notebook, we'll use `pl.GPUEngine`. The default configuration has transparent CPU fallback for unsupported operations, so if we execute an unsupported query we don't get an error. To prove we're running on the GPU, we'll pass a configured engine object that will raise an error if we can't run the query.

_If you're running with [jupyterlab-nvdashboard](https://developer.nvidia.com/blog/maximize-gpu-performance-with-near-real-time-usage-stats-on-nvdashboard-v0-10/) you should see the GPU Memory and Utilization tick up._

In [ ]:
gpu_engine = pl.GPUEngine(
    device=0, # This is the default
    raise_on_fail=True, # Fail loudly if we can't run on the GPU.
)

The very first collection on the GPU will take a couple of seconds. The GPU engine is lazy-loaded so that even if the necessary packages are installed, Polars' fast import times are not affected. Consequently, when we trigger GPU execution for the first time, we load a number of additional packages, and initialize GPU-specific data structures and contexts.

In [ ]:
transactions.select(pl.col("AMOUNT").sum()).collect(engine=gpu_engine)

We probably don't need a GPU for such a simple operation on this dataset. But when we start doing more complex analysis, the high-bandwidth memory and compute power of GPUs will make things much more interactive.

# More Complex Analysis

While the data is synthetic, it's representative of the kinds of datasets that come up in financial services, retail/e-commerce, consumer internet, and other industries.

With this data, we can see how using GPU-accelerated Polars provides a significant productivity boosts by exploring common business questions like:

1. Which customers have the largest total transctions?
2. What's the per-month transaction amount for each spending category over time?
3. How does weather effect the transactions for each category?

Because this data is synthentic, the results won't be meaningful in terms of business conclusions. Rather, we are motivating _why_ using this GPU engine is useful in terms of perfomance and cost reduction. We'll prove that the CPU and GPU engine get the same results in the first example, but as we move throughout the notebook we'll take this for granted and focus on CPU and GPU timings.

Let's start with our first question.

## Which customers have the largest total transactions?

In [ ]:
%%time

res_cpu = (
    transactions
    .group_by("CUST_ID")
    .agg(pl.col("AMOUNT").sum())
    .sort(by="AMOUNT", descending=True)
    .head()
    .collect()
)
res_cpu

In [ ]:
%%time

res_gpu = (
    transactions
    .group_by("CUST_ID")
    .agg(pl.col("AMOUNT").sum())
    .sort(by="AMOUNT", descending=True)
    .head()
    .collect(engine=gpu_engine)
)
res_gpu

In [ ]:
assert_frame_equal(res_cpu, res_gpu)

Great! We see a nice performance gain when using the GPU engine and we've confirmed the results are equal.

In addition to the Dataframe interface, Polars also has an SQL interface. We can also use this with the GPU engine, since Polars translates both the DataFrame and SQL interfaces into a query execution plan.

In [ ]:
query = """
SELECT CUST_ID, SUM(AMOUNT) as sum_amt
FROM transactions
GROUP BY CUST_ID
ORDER BY sum_amt desc
LIMIT 5
"""

%time pl.sql(query).collect()
%time pl.sql(query).collect(engine=gpu_engine)

## Which customers have the largest single transaction?

Customer `CP2KXQSX9I` had the largest total transactions over time (on the full dataset), but they might not have the largest single transaction. Let's find the top customers by single transaction amount.

In [ ]:
%%time

(
    transactions
    .group_by("CUST_ID")
    .agg(pl.col("AMOUNT").max().alias("max_amount"))
    .sort(by="max_amount", descending=True)
    .head()
    .collect()
)

In [ ]:
%%time

(
    transactions
    .group_by("CUST_ID")
    .agg(pl.col("AMOUNT").max().alias("max_amount"))
    .sort(by="max_amount", descending=True)
    .head()
    .collect(engine=gpu_engine)
)

Again, we see a nice speedup using the GPU engine.

## Investigating a specific customer

Let's explore the transactions of a single customer now. What was `CIP0I11MG2`'s largest transaction?

In [ ]:
%%time

(
    transactions
    .filter(pl.col("CUST_ID") == "CIP0I11MG2")
    .select(pl.col("AMOUNT").max())
    .collect()
)

In [ ]:
%%time

(
    transactions
    .filter(pl.col("CUST_ID") == "CIP0I11MG2")
    .select(pl.col("AMOUNT").max())
    .collect(engine=gpu_engine)
)

### Sidebar: Why was performance using the CPU and GPU engines similar for this query?

What we've seen so far is that, for many common queries, we see significant performance increases when we use the GPU. But for this last one, things were more similar.

Using Polars on a GPU isn't **always** a surefire winner in terms of speed compared to Polars on a CPU. For simple queries that aren't computationally heavy, like the last query, we're often limited by the speed that we can read results from disk.

Let's confirm this by profiling.

In [ ]:
res, prof = (
    transactions
    .filter(pl.col("CUST_ID") == "CIP0I11MG2")
    .select(pl.col("AMOUNT").max())
    .profile()
)

prof.with_columns(
    ((pl.col("end") - pl.col("start")) / pl.col("end").max() * 100)
    .alias("pct_time_spent")
)

We spent 99.9%+ of the time just reading the data. Polars can use the GPU-accelerated Parquet reader to read this data, but ultimately when we're **IO bound** like this, there's less opportunity for GPU-acceleration.

Let's try an even more computationally intense query.

## What's the per-month transaction amount for each category over time?

For this query, we'll group and sort down to the individual month, which takes more work.

In [ ]:
%%time

res = (
    transactions
    .group_by(["EXP_TYPE", "YEAR", "MONTH"])
    .agg(pl.mean("AMOUNT"))
    .sort(["EXP_TYPE", "YEAR", "MONTH"])
    .collect()
)

In [ ]:
%%time

res = (
    transactions
    .group_by(["EXP_TYPE", "YEAR", "MONTH"])
    .agg(pl.mean("AMOUNT"))
    .sort(["EXP_TYPE", "YEAR", "MONTH"])
    .collect(engine=gpu_engine)
)

Again, since this query does more work we see strong performance benefits on the GPU.

## Interoperability

As mentioned in the introduction, all results will be materialized as standard CPU Polars DataFrames.

That means that any results from GPU-accelerated queries interoperate seamlessly with the rest of the polars ecosystem. For example, we can use Polars' built-in plotting functionality on the result we created from the GPU engine.

**Colab Note**: Using HoloViews/Bokeh in Colab is not fully supported, so plots may not render correctly.

In [ ]:
res = res.with_columns(
    pl.datetime(pl.col("YEAR"), pl.col("MONTH"), day=1)
    .alias("year-mon")
)

res.plot.scatter(x="year-mon", y="AMOUNT", groupby="EXP_TYPE")

Interoperability goes in the other direction, too. We can use materialized CPU DataFrames in downstream operations on the GPU. Polars will handle any necessary CPU/GPU conversions for us, which we'll see shortly.

# Moving Beyond Our Single Dataset

Let's bring in some weather data to evaluate if transactions are impacted by weather. For example, spending behavior for certain categories could be influenced by whether it's raining or even _how much_ it's raining.

Like our transactions dataset, here we'll use a simulated rainfall dataset.

## Is more rain related to more transactions across specific categories?

Our rainfall dataset is a CSV file, so we'll use the CSV reader here and take a look at a few rows.

In this case, we'll load the data using the CPU to highlight how smoothly Polars handles mixing CPU and GPU objects.

In [ ]:
names = ['Location', 'Rainfall (inches)', 'Date', 'YEAR', 'MONTH', 'DAY']

weather = pl.scan_csv("rainfall_data_2010_2020.csv", new_columns=names)
weather.head().collect()

Data is often messy. We've got dates in this CSV file, but they weren't parsed as dates by the CSV reader due to the format. We can easily clean this up by typecasting, after which we'll materialize the DataFrame in CPU memory.

In [ ]:
weather_cleaned = (
    weather
    .with_columns(pl.col("Date").cast(pl.Utf8).str.strptime(pl.Date(), "%Y%m%d"))
    .collect()
)

To answer the original question about rainfall and spending, we need to join these datasets together.

We can pass this CPU in-memory DataFrame to a Polars Lazy API query by calling `.lazy()`.

In [ ]:
%%time

(
    transactions
    .join(
        other=weather_cleaned.lazy(),
        left_on="DATE",
        right_on="Date",
        how="inner"
    )
    .group_by(["EXP_TYPE", "DATE"])
    .agg(pl.mean("Rainfall (inches)"))
    .sort(["DATE", "EXP_TYPE", "Rainfall (inches)"])
    .head()
    .collect()
)

When we run the same query on the GPU, Polars handles the CPU / GPU transfers for us under the hood.

In [ ]:
%%time

(
    transactions
    .join(
        other=weather_cleaned.lazy(),
        left_on="DATE",
        right_on="Date",
        how="inner"
    )
    .group_by(["EXP_TYPE", "DATE"])
    .agg(pl.mean("Rainfall (inches)"))
    .sort(["DATE", "EXP_TYPE", "Rainfall (inches)"])
    .head()
    .collect(engine=gpu_engine)
)

Let's demonstrate some more of the native plotting facilities in Polars by answering the question in more holistic fashion.

We'll make a plot of total rainfall and total amount spent per month for each year and each type of expenditure.

In [ ]:
res = (
    transactions
    .join(
        other=weather_cleaned.lazy(),
        left_on="DATE",
        right_on="Date",
        how="inner"
    )
    .group_by(["EXP_TYPE", "YEAR", "MONTH"])
    .agg(pl.sum("Rainfall (inches)"), pl.sum("AMOUNT"))
    .sort(["YEAR", "MONTH"])
    .collect(engine=gpu_engine)
)

res

In [ ]:
(
    res
    .with_columns(
        pl.date(pl.col("YEAR"), pl.col("MONTH"), 1).alias("date-month"),
        pl.col("Rainfall (inches)")*100,
    )
    .plot.line(
        x="date-month", y=["AMOUNT", "Rainfall (inches)"],
        groupby=['EXP_TYPE'],
        rot=45,
    )
)

We intially constructed the GPU engine with `raise_on_fail=True` to demonstrate the breadth of GPU support. Now, we'll demonstrate how a not yet GPU supported feature is handled gracefully when we set `raise_on_fail=False` (the default behavior for `engine="gpu"`).

When we use `rolling_mean`, our query smoothly falls back to the CPU to continue running rather than failing.

In [ ]:
engine_with_fallback = pl.GPUEngine(
    device=0, # This is the default
    raise_on_fail=False, # Fallback to CPU if we can't run on the GPU (this is the default)
)

In [ ]:
%%time

result = transactions.with_columns(
    pl.col('AMOUNT').rolling_mean(
        window_size=7
    )
     .alias('rolling_avg')
).collect(engine=engine_with_fallback)

result.head()

# Conclusion

With the new Polars GPU engine powered by cuDF, you can potentially get significant performance gains and lower costs for workflows processing 10-100+ GB of data.

To learn more, we encourage you to visit the Polars GPU Support documentation or visit rapids.ai/cudf-polars.